In [1]:
! pip install pandas
! pip install fake_useragent 
! pip install tqdm

In [2]:
import requests
import pandas as pd
import time as time
from datetime import datetime
from fake_useragent import UserAgent
from tqdm import tqdm
import os
import math

In [3]:
def read_html_thing():
    df1 = pd.DataFrame()
    
    urlBase = 'https://twitchtracker.com/channels/ranking?page='
    
    ua = UserAgent()
    
    header = {
      "User-Agent": ua.random,
      "X-Requested-With": "XMLHttpRequest"
    }
    
    try:
        for page in range(76, 200):
            url = urlBase + str(page)
            time.sleep(1)

            r = requests.get(url, headers=header)

            dfs = pd.read_html(r.text)
            unclean = dfs[0]

            df2 = pd.DataFrame()

            df2['Name'] = list(filter(lambda x: '.ads ' not in x, unclean['Unnamed: 2'].to_list()))
            df2['Avg Viewers'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['AVG Viewers'].to_list()))))
            df2['Time Streamed'] = list(map(float, list(map(lambda x: x.replace('hours', ''), list(filter(lambda x: '.ads ' not in x, unclean['Time Streamed'].to_list()))))))
            df2['All Time Peak Viewers'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['All Time Peak Viewers'].to_list()))))
            df2['Hours Watched'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['Hours Watched'].to_list()))))
            df2['Rank'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['Rank'].to_list()))))
            df2['Followers Gained'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['Followers Gained'].to_list()))))
            df2['Total Followers'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['Total Followers'].to_list()))))
            df2['Total Views'] = list(map(int, list(filter(lambda x: '.ads ' not in x, unclean['Total Views'].to_list()))))

            df1 = pd.concat([df2, df1], ignore_index=True)
    except:
        print(f"Stopped")
        
    #can sort by changing the title below from name to Total Views
    #default is rank
    return df1.sort_values('Rank').reset_index(drop=True) 

# df = read_html_thing()
# df.to_excel("General_Info3.xlsx")

In [4]:
# Time streamed is in Hours
# Avg viewers, Time streamed, hours watched, followers gained are for the last 30 days
#By default it is sorted by rank, which take into account on average concurrent viewers, followers, views and stream time for the last 30 days

df = pd.read_excel("General_Info.xlsx")

# for index in range(1, 4):
#     df1 = pd.read_excel(f"General_Info{index}.xlsx")
#     df = pd.concat([df1, df])
    
# # df.head()
# df.count()
# df.to_excel("General_info_final.xlsx")

In [5]:
df.Name = df.Name.replace('[^0-9a-zA-Z.,-/ ]', ' ', regex=True)
df = df[df.Name.str.isalnum()]
df = df[df.Name.str.strip() != '']
nameList = df.Name.to_list()
nameList


['xQcOW',
 'ibai',
 'auronplay',
 'Asmongold',
 'Sapnap',
 'Gaules',
 'galaxyracerdxb',
 'LEC',
 'LCK',
 'Fextralife',
 'n3koglai',
 'AdinRoss',
 'Trainwreckstv',
 'RocketLeague',
 'Maximum',
 'HasanAbi',
 'casimito',
 'tommyinnit',
 'Mizkif',
 'ROSHTEIN',
 'ElMariana',
 'juansguarnizo',
 'Rubius',
 'VALORANT',
 'eliasn97',
 'tarik',
 'Elraenn',
 'IlloJuan',
 'NICKMERCS',
 'RanbooLive',
 'ElSpreen',
 'LCS',
 'aminematue',
 'summit1g',
 'WilburSoot',
 'ImKaiCenat',
 'elxokas',
 'MontanaBlack88',
 'CBLOL',
 'Tubbo',
 'buster',
 'loltyler1',
 'stylishnoob4',
 'Faker',
 'TheGrefg',
 'deepins02',
 'DreamLeague',
 'TrashTastePodcast',
 'TeamLiquid',
 'aceu',
 'Kamet0',
 'Evelone192',
 'buddha',
 'CriticalRole',
 'sodapoppin',
 'TenZ',
 'LVPes',
 'Sykkuno',
 'Elajjaz',
 'egorkreed',
 'QTCinderella',
 'LyonWGFLive',
 'AustinShow',
 'Trymacs',
 'bratishkinoff',
 'HitPointOne',
 'LIRIK',
 'JojoHF',
 'shroud',
 'Clix',
 'alanzoka',
 'GRONKH',
 'Rainbow6',
 'BruceDropEmOff',
 'EintrachtSpandau',
 

In [6]:
# namesPD=pd.DataFrame()
# namesPD=df.Name.to_list()
# namesPD
# namesPD.columns = ['login']
# namesPD.to_excel("New_Names.xlsx")
# names = df.Name.to_list()
# len(names)

In [7]:
import math

Client_ID = 'eceifr3vixcltzyw8g4f9yeiv0rpy8'
Secret  = 'vxt336xddant33lagzkvpwo5n5ibw5'

def urlSearch(URL, nameList, count, paramStr):
    
    for x in nameList[100*(count-1) : (100*count)]:
        if URL[-1] == '?':
            URL = f"{URL}{paramStr}={x}"
        else:
            URL =  f"{URL}&{paramStr}={x}"

    return URL

def Check(endpoint, paramStr, nameList, countsOf100):
    
    autCallJson = requests.post(url='https://id.twitch.tv/oauth2/token', 
                                params={'client_id': Client_ID,
                                        'client_secret': Secret,
                                        'grant_type': 'client_credentials'}).json()
    
    head = {'Client-ID' : Client_ID,
            'Authorization' :  "Bearer " + autCallJson['access_token']}
    
    df = pd.DataFrame()

    urlEndpoint = f"https://api.twitch.tv/helix/{endpoint}?"
    
    for index in range(1, countsOf100+1):
        rJson = requests.get(urlSearch(urlEndpoint, 
                                       nameList, 
                                       index, 
                                       paramStr), 
                         headers = head).json()
        df = pd.concat([df, pd.json_normalize(rJson['data'])], 
                       ignore_index=True)

    return df

def addCurrentHrStreams(nameList, noOfStreamersFromList):
    
    countOf100 = int(math.ceil(noOfStreamersFromList / 100.0))
    
    df1 = Check('streams', 'user_login', nameList, countOf100)
    df1 = df1.drop(columns=['game_id', 'thumbnail_url', 'tag_ids'])
    df1['time_extracted'] = datetime.now()
    
    return df1
    
def mainFunction(path, durationInHours=1, nameList=nameList, topNoOfStreamers=1000):
    
    if os.path.exists(path):
        final = pd.read_excel(path, index_col=0)
    else:
        final = pd.DataFrame()
    
    for x in tqdm(range(durationInHours)):
        df1 = addCurrentHrStreams(nameList, topNoOfStreamers)
        df1.index = df1.index + 1
        final = pd.concat([final, df1], ignore_index=False)
        final.to_excel(path)
        time.sleep(3590)

mainFunction("Live_Stream_Data.xlsx", 8)

100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [7:59:09<00:00, 3593.71s/it]


In [8]:
# https://api.twitch.tv/helix/subscriptions
# GET https://api.twitch.tv/helix/users/follows?to_id=<user ID>

# df3 = namesPD.merge(df, how='left', indicator=True)

# df3 = df3[df3['_merge'] == 'left_only']
# df3


# # df3.count()

# df1 = Check()

# # df1 = df1.drop(columns=['game_id', 'thumbnail_url', 'tag_ids'])

# df1['time_extracted'] = datetime.now()

# df1


# import json

# f = open ('jsonviewer.json', "r")
# ff = json.load(f)

# dffff = pd.DataFrame()

# # print(ff['data'][0])
# for x in ff['data']:
#     dffff = pd.concat([dffff, pd.DataFrame(x)])
    
# dffff.head()


In [9]:
# def urlSearch(count):
    
#     URL = 'https://api.twitch.tv/helix/streams?'
    
#     for x in nameList[100*(count-1) : (100*count)+1]:
#         if URL[-1] == '?':
#             URL = URL + 'user_login=' + x
#         else:
#             URL = URL + '&user_login=' + x
            
            
#     return URL
        
# authURL = 'https://id.twitch.tv/oauth2/token'
# Client_ID = 'eceifr3vixcltzyw8g4f9yeiv0rpy8'
# Secret  = 'vxt336xddant33lagzkvpwo5n5ibw5'

# AutParams = {'client_id': Client_ID,
#              'client_secret': Secret,
#              'grant_type': 'client_credentials'
#              }


# def Check():
#     AutCall = requests.post(url=authURL, params=AutParams) 
#     access_token = AutCall.json()['access_token']

#     head = {
#     'Client-ID' : Client_ID,
#     'Authorization' :  "Bearer " + access_token
#     }
    
#     df = pd.DataFrame()

#     for x in range(1, 21):
#         r = requests.get(urlSearch(x), headers =  head).json()
        
#         df = pd.concat([df, pd.json_normalize(r['data'])], ignore_index=True)
        
           
#     return df


# final = pd.DataFrame()

# count = 0


# def hourlyFunction(final, count):
#     df1 = Check()
#     df1 = df1.drop(columns=['game_id', 'thumbnail_url', 'tag_ids'])
#     df1['time_extracted'] = datetime.now()
#     time.sleep(3590)
#     final = pd.concat([final, df1], ignore_index=False)
#     count = count+1
    
#     if count <= 12:
#         hourlyFunction(final, count)
#     else:
#         final.index = final.index + 1
#         final.to_excel("Live_Stream_Data.xlsx")
        
# hourlyFunction(final, count)

    

In [10]:
# def urlSearch(count):
    
#     URL = 'https://api.twitch.tv/helix/users?'
    

#     for x in names[100*(count-1)+1 : (100*count)+1]:
#         if URL[-1] == '?':
#             URL = URL + 'login=' + x
#         else:
#             URL = URL + '&login=' + x
            
    
            
#     return URL
        
# authURL = 'https://id.twitch.tv/oauth2/token'
# Client_ID = 'eceifr3vixcltzyw8g4f9yeiv0rpy8'
# Secret  = 'vxt336xddant33lagzkvpwo5n5ibw5'

# AutParams = {'client_id': Client_ID,
#              'client_secret': Secret,
#              'grant_type': 'client_credentials'
#              }


# def Check():
#     AutCall = requests.post(url=authURL, params=AutParams) 
#     access_token = AutCall.json()['access_token']

#     head = {
#     'Client-ID' : Client_ID,
#     'Authorization' :  "Bearer " + access_token
#     }
    
#     df = pd.DataFrame()

#     for x in range(1, 11):
#         r = requests.get(urlSearch(x), headers = head).json()
#         if r['data']: 
#             df = pd.concat([df, pd.json_normalize(r['data'])],ignore_index=True)
#             df.index = df.index + 1
    
#     return df

# df = Check().drop(columns=['profile_image_url', 'offline_image_url', 'type'])
# df = df.groupby('id').filter(lambda x : x['id'].shape[0] < 2)

# df

# df.to_excel("All_1000_users_general_information.xlsx")

In [11]:

# idList = df.id.to_list()

# def urlSearch(id):

#     URL = 'https://api.twitch.tv/helix/users/follows?to_id='+id
            
#     return URL
        
#     authURL = 'https://id.twitch.tv/oauth2/token'
#     Client_ID = 'eceifr3vixcltzyw8g4f9yeiv0rpy8'
#     Secret  = 'vxt336xddant33lagzkvpwo5n5ibw5'

#     AutParams = {'client_id': Client_ID,
#                  'client_secret': Secret,
#                  'grant_type': 'client_credentials'
#                  }

# def Check():
#     AutCall = requests.post(url=authURL, params=AutParams) 
#     access_token = AutCall.json()['access_token']

#     head = {
#     'Client-ID' : Client_ID,
#     'Authorization' :  "Bearer " + access_token
#     }
    
#     df = pd.DataFrame()

#     for x in idList:
#         r = requests.get(urlSearch(x), headers = head).json()
#         df = pd.concat([df, pd.DataFrame({'to': [r['data'][0]['to_name']], 'total':[r['total']]})],ignore_index=True)
#         df.index = df.index + 1

#     return df


# df2 = Check().drop_duplicates()

# df2.to_excel("Streamers_follower_count.xlsx")

In [12]:
# def read_list(file):
#     channel_list = []
#     with open(file, 'r', encoding='utf-8') as filehandler:
#         channel_list = filehandler.readlines()
#     channel_list = [streamer[:-1] for streamer in channel_list]
#     return channel_list

# names = read_list("names.txt")

# namesPD = pd.DataFrame(names)

# namesPD = namesPD[1:] #take the data less the header row
# namesPD.columns = ['login'] #set the header row as the df header

# namesPD = namesPD.sort_values(by=['login'])

# namesPD.login = namesPD.login.apply(lambda x: x.lower())

# namesPD